In [30]:
import pandas as pd
import sklearn
import numpy as np

In [31]:
from sklearn.ensemble import IsolationForest

## Method Testing

In [32]:
sales = pd.read_csv('sales_mod.csv')

/tmp/ipykernel_1086328/1754691465.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv('sales_mod.csv')


In [33]:
sales.drop(columns=['Unnamed: 0'], inplace=True)

In [34]:
sales.set_index('pin', inplace=True)

In [35]:
sales.columns

Index(['township_code', 'class', 'year', 'sale_date', 'sale_price',
       'sale_price_log10', 'sale_key', 'doc_no', 'deed_type', 'seller_name',
       'is_multisale', 'num_parcels_sale', 'buyer_name', 'sale_type',
       'sale_filter_lower_limit', 'sale_filter_upper_limit',
       'sale_filter_count', 'property_advertised',
       'is_installment_contract_fulfilled',
       'is_sale_between_related_individuals_or_corporate_affiliates',
       'is_transfer_of_less_than_100_percent_interest',
       'is_court_ordered_sale', 'is_sale_in_lieu_of_foreclosure',
       'is_condemnation', 'is_short_sale', 'is_bank_reo_real_estate_owned',
       'is_auction_sale', 'is_seller_buyer_a_relocation_company',
       'is_seller_buyer_a_financial_institution_or_government_agency',
       'is_buyer_a_real_estate_investment_trust', 'is_buyer_a_pension_fund',
       'is_buyer_an_adjacent_property_owner',
       'is_buyer_exercising_an_option_to_purchase',
       'is_simultaneous_trade_of_property', 'is_s

In [36]:
feed_data = sales[['sale_price_log10', 'price_per_sqft_log10', 'pct', 'days_since_last_transaction', 'counts', 'town_class_movement', ]]

In [37]:
feed_data.fillna(0, inplace=True)

/tmp/ipykernel_1086328/719415807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feed_data.fillna(0, inplace=True)


In [38]:
feed_data

,sale_price_log10,price_per_sqft_log10,pct,days_since_last_transaction,counts,town_class_movement
pin,,,,,,
2092050370000,5.303196,0.004592,0.0,0.0,0.0,0.0
20311070380000,5.332438,0.003377,0.0,0.0,0.0,0.0
10201120410000,5.732394,0.002541,0.0,0.0,0.0,0.0
16183030040000,5.774517,0.003231,0.0,0.0,0.0,0.0
10272320440000,5.454845,0.004095,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
3081110200000,5.742823,0.002749,0.0,0.0,0.0,0.0
25212060060000,4.785330,0.005213,0.0,0.0,0.0,0.0
7294030160000,5.342423,0.002293,0.0,0.0,0.0,0.0


In [39]:
isof = IsolationForest()
prediction = isof.fit_predict(feed_data)

In [40]:
# -1 is outlier and 1 is normal
# Looking at human readable CSV, this looks to be about similar to (1,2) in terms of outlier detections just from raw numbers.
np.unique(prediction, return_counts=True)

(array([-1,  1]), array([ 33691, 184448]))